In [1]:
from random import *
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import PIL
import tensorflow as tf
import numpy as np
from keras import layers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.optimizers import SGD
from keras import Sequential
import math
import sys
import gc


In [2]:
tf.random.set_seed(960312)

number_of_packer_to_drop = 1
path = "/media/2/Network/extracted_feature/whole_shuffle_to_19/with_error_whole/"
training_result_file =  "result_back_1_adam.txt"
back_layer_file = "/media/2/Network/pretrained_model/back_layers_19~.h5"
val_lr = [5e-05, 1e-06]
val_decay = [0, 1e-3, 1e-6]
val_amsgrad = [False,True]

test_data = np.load(path+"testing_features.npy",mmap_mode="r")
test_label = np.load(path+"testing_label.npy",mmap_mode="r")

In [16]:
def packet_drop(arr,number_of_packer_to_drop):
    index = randrange(65-number_of_packer_to_drop)
    arr[:,:,index*8:(index*8+8*number_of_packer_to_drop)] = 0
def error_injection(data,number_of_packer_to_drop):
    for img in data:
        packet_drop(img,number_of_packer_to_drop)

In [11]:
back_layer = load_model(back_layer_file)
back_layer.layers[1].trainable = True
for i in range(2,4):
    back_layer.layers[i].trainable = False
    #optimizer = tf.keras.optimizers.Adagrad(lr=val_lr, decay=0)
optimizer = tf.keras.optimizers.Adam(lr=1e-06,beta_1=0.9,beta_2=0.999,epsilon=None, decay=1e-6,amsgrad=True)
back_layer.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
test_result =back_layer.evaluate(test_data,test_label)
print("with packet error result == loss :{:.4}, Acc : {:.4}% ".format(test_result[0], test_result[1]*100))

1563/1563 [==============================] - 9s 6ms/step - loss: 2.7649 - accuracy: 0.6558
with packet error result == loss :2.765, Acc : 65.58% 


In [6]:
!ls /media/2/Network/extracted_feature/whole_shuffle_to_19/

testing_features.npy  train_features_400000.npy  train_label_400000.npy
testing_label.npy     train_features_800000.npy  train_label_800000.npy
train_features_0.npy  train_label_0.npy		 with_error_whole


In [7]:
original_path = "/media/2/Network/extracted_feature/whole_shuffle_to_19/"
test_data_original = np.load(original_path+"testing_features.npy",mmap_mode="r")
test_label_original = np.load(original_path+"testing_label.npy",mmap_mode="r")

In [12]:
test_result =back_layer.evaluate(test_data_original,test_label_original)
print("original packet result == loss :{:.4}, Acc : {:.4}% ".format(test_result[0], test_result[1]*100))

1563/1563 [==============================] - 9s 6ms/step - loss: 2.7567 - accuracy: 0.6589
original packet result == loss :2.757, Acc : 65.89% 


In [17]:
for i in range(1,11):
    test_data = np.load(original_path+"testing_features.npy")
    test_label = np.load(original_path+"testing_label.npy")
    error_injection(test_data,i)
    test_result =back_layer.evaluate(test_data,test_label)
    print(i,"of packet error result == loss :{:.4}, Acc : {:.4}% ".format(test_result[0], test_result[1]*100))

1563/1563 [==============================] - 9s 6ms/step - loss: 2.7685 - accuracy: 0.6562
1 of packet error result == loss :2.768, Acc : 65.62% 
1563/1563 [==============================] - 9s 6ms/step - loss: 2.7751 - accuracy: 0.6526
2 of packet error result == loss :2.775, Acc : 65.26% 
1563/1563 [==============================] - 9s 6ms/step - loss: 2.7942 - accuracy: 0.6469
3 of packet error result == loss :2.794, Acc : 64.69% 
1563/1563 [==============================] - 9s 6ms/step - loss: 2.8108 - accuracy: 0.6429
4 of packet error result == loss :2.811, Acc : 64.29% 
1563/1563 [==============================] - 10s 7ms/step - loss: 2.8293 - accuracy: 0.6401
5 of packet error result == loss :2.829, Acc : 64.01% 
1563/1563 [==============================] - 9s 6ms/step - loss: 2.8463 - accuracy: 0.6367
6 of packet error result == loss :2.846, Acc : 63.67% 
1563/1563 [==============================] - 9s 6ms/step - loss: 2.8528 - accuracy: 0.6307
7 of packet error result == loss